# QOSF Cohort 10
## Task 3

In [73]:
!pip install docplex
!pip install dwave-ocean-sdk
!pip install qiskit qiskit-aer qiskit-machine-learning
!pip install qiskit-aer
!pip install qiskit qiskit-aer qiskit-ignis


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for numpy<2.0.0,>=1.17.3 from https://files.pythonhosted.org/packages/bc/40/d6f7ba9ce5406b578e538325828ea43849a3dfd8db63d1147a257d19c8d1/numpy-1.19.5-cp39-cp39-win_amd64.whl.metadata
  Using cached numpy-1.19.5-cp39-cp39-win_amd64.whl.metadata (2.0 kB)
Using cached numpy-1.19.5-cp39-cp39-win_amd64.whl (13.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2


ERROR: Cannot uninstall numpy 1.25.2, RECORD file not found. You might be able to recover from this via: 'pip install --force-reinstall --no-deps numpy==1.25.2'.

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'c:\\users\\jerom\\appdata\\local\\programs\\python\\python39\\lib\\site-packages\\numpy-1.25.2.dist-info\\METADATA'


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'c:\\users\\jerom\\appdata\\local\\programs\\python\\python39\\lib\\site-packages\\numpy-1.25.2.dist-info\\METADATA'


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'c:\\users\\jerom\\appdata\\local\\programs\\python\\python39\\lib\\site-packages\\numpy-1.25.2.dist-info\\METADATA'


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [79]:
import numpy as np
np.bool = np.bool_
from docplex.mp.model import Model
from qiskit.circuit.library import EfficientSU2
from qiskit.circuit import QuantumCircuit
from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit.circuit import Parameter
from qiskit.primitives import Sampler
#from qiskit.algorithms.optimizers import SLSQP
#from qiskit.circuit.library import Z2Symmetries
#from qiskit.algorithms import VQE
from qiskit.quantum_info import Pauli
#from dwave.system import DWaveSampler, EmbeddingComposite




1. From Integer Linear Programming (ILP) to Quadratic Unconstrained Binary Optimization (QUBO)

* Define the ILP formulation of the BPP. You can use Docplex or similar frameworks to do it. 



The ILP formulation of the BPP is : 

**Variables :**

* binary variables : $x_{i,j}=1$ if item i is placed in bin j, 0 otherwise
* binary variables : $y_j=1$ if bin j is used 0 otherwise

**Objective Function :**

* minimize $\sum_j y_j$

**Constraints:**

* $\forall i, j, x_{i,j}, y_j \in \left\{0,1\right\} $
* $\sum_jx_{i,j}=1, \forall i$ : each item must be placed in exactly one bin
* $\sum_iw_ix_{i,j}\leq Wy_j, \forall j$ : The total weight of items in each bin must not exceed the bin capacity W


In [80]:
# The ILP formultation is
# docplex model
from docplex.mp.model import Model

def create_ilp(weights, capacity):
    # Initialize the model
    model = Model('bin_packing')
    
    num_items = len(weights)
    # Create binary variables for items in bins
    x = model.binary_var_matrix(num_items, num_items, name='x')  # x[i][j] = 1 if item i is in bin j
    y = model.binary_var_list(num_items, name='y')  # y[j] = 1 if bin j is used

    # Objective: Minimize the number of bins used
    model.minimize(model.sum(y[j] for j in range(num_items)))

    # Constraint 1: Each item must be placed in exactly one bin
    for i in range(num_items):
        model.add_constraint(model.sum(x[i, j] for j in range(num_items)) == 1)

    # Constraint 2: The total weight in each bin must not exceed the capacity
    for j in range(num_items):
        model.add_constraint(model.sum(weights[i] * x[i, j] for i in range(num_items)) <= capacity * y[j])

    return model

* Create a function to transform the ILP model into a QUBO 

To Transform the ILP model into a QUBO, we need to express the objaectived functions and the constraint in a quadratic form. To do so we keep the objective functions and reformulate constraints in term of pernalties with slack variables : 

Cosntraint : 
$$Q(x,y) = \sum_j y_j + \sum_{i,j}\left(1-\sum_jx_{i,j}\right)^2+\sum_jmax\left(0,\sum_iw_ix_{i,j}-Wy_j\right)^2$$

Objective 

In [81]:
def ilp_to_qubo(weights, capacity):
    num_items = len(weights)
    num_bins = num_items  # At most one bin per item

    qubo_matrix = np.zeros((num_items + num_bins, num_items + num_bins))

    # Objective: Minimize the number of bins used
    for j in range(num_bins):
        qubo_matrix[num_items + j, num_items + j] += 1  # Coefficient for y_j

    # Constraint: Each item must be assigned to exactly one bin
    for i in range(num_items):
        for j in range(num_bins):
            qubo_matrix[i, i] += 1  # Self-penalty for item i
            qubo_matrix[i, num_items + j] += -2  # Coupling between x_ij and y_j

    # Constraint: Total weight in each bin should not exceed capacity
    for j in range(num_bins):
        for i in range(num_items):
            weight_term = weights[i]
            qubo_matrix[num_items + j, num_items + j] += weight_term**2  # Penalty for excess weight
            qubo_matrix[i, num_items + j] += -weight_term  # Coupling term

    return qubo_matrix

* Test your function with specific instances (size small, medium, and big)

* Small Instance : 3 items with a maximum capacity of 4
* Medium Instance : 5 items and maximum capacity of 10 
* LArge instance : 10 items and maximum capacity of 20

In [82]:
test_cases = {
    "Small": ([1, 2, 3], 4),               # 3 items, capacity 4
    "Medium": ([2, 5, 3, 7, 1], 10),      # 5 items, capacity 10
    "Large": ([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 20)  # 10 items, capacity 20
}

# Run tests
for name, (weights, capacity) in test_cases.items():
    print(f"\nTesting {name} Instance:")
    qubo_matrix = ilp_to_qubo(weights, capacity)
    print("QUBO Matrix:")
    print(qubo_matrix)


Testing Small Instance:
QUBO Matrix:
[[ 3.  0.  0. -3. -3. -3.]
 [ 0.  3.  0. -4. -4. -4.]
 [ 0.  0.  3. -5. -5. -5.]
 [ 0.  0.  0. 15.  0.  0.]
 [ 0.  0.  0.  0. 15.  0.]
 [ 0.  0.  0.  0.  0. 15.]]

Testing Medium Instance:
QUBO Matrix:
[[ 5.  0.  0.  0.  0. -4. -4. -4. -4. -4.]
 [ 0.  5.  0.  0.  0. -7. -7. -7. -7. -7.]
 [ 0.  0.  5.  0.  0. -5. -5. -5. -5. -5.]
 [ 0.  0.  0.  5.  0. -9. -9. -9. -9. -9.]
 [ 0.  0.  0.  0.  5. -3. -3. -3. -3. -3.]
 [ 0.  0.  0.  0.  0. 89.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. 89.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. 89.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0. 89.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0. 89.]]

Testing Large Instance:
QUBO Matrix:
[[ 10.   0.   0.   0.   0.   0.   0.   0.   0.   0.  -3.  -3.  -3.  -3.
   -3.  -3.  -3.  -3.  -3.  -3.]
 [  0.  10.   0.   0.   0.   0.   0.   0.   0.   0.  -4.  -4.  -4.  -4.
   -4.  -4.  -4.  -4.  -4.  -4.]
 [  0.   0.  10.   0.   0.   0.   0.   0.   0.   0.  -5.  -5.  -5.  -5.
   -5.  -

2. Create a Brute Force solver for the QUBO problem and solve the specific instances

The brute force solver works in the following way : 
* we go through all possible assignment where each items can go into any of the available bins
* we keep track of the total weight in each bin and ccheck that the total weight does not exceed the capacity, if it does the assignment is considered as not valid and it is skipped
* the optimal solutions is chosen amond the valid assignment by picking the one that uses the less bin
* the retuern is the best assignment and the number of bins used

In [83]:
import itertools

def brute_force_qubo(weights, capacity):
    num_items = len(weights)
    best_solution = None
    best_bin_count = float('inf')

    # Generate all possible assignments of items to bins
    for assignment in itertools.product(range(num_items), repeat=num_items):
        # Initialize total weights for each bin
        total_weights = [0] * num_items
        # Use a set to track which bins are used
        bins_used = set()

        # Calculate total weight per bin and track used bins
        for item_index, bin_index in enumerate(assignment):
            total_weights[bin_index] += weights[item_index]
            bins_used.add(bin_index)  # Mark bin as used

        # Check if any bin exceeds the capacity
        if all(weight <= capacity for weight in total_weights):
            current_bin_count = len(bins_used)
            # Update best solution if this one uses fewer bins
            if current_bin_count < best_bin_count:
                best_bin_count = current_bin_count
                best_solution = assignment

    return best_solution, best_bin_count

In [ ]:

# Run brute force solver for each instance
for name, (weights, capacity) in test_cases.items():
    print(f"\nTesting {name} Instance:")
    solution, bin_count = brute_force_qubo(weights, capacity)
    print(f"Best assignment: {solution}")
    print(f"Minimum number of bins used: {bin_count}")


Testing Small Instance:
Best assignment: (0, 0, 1)
Minimum number of bins used: 2

Testing Medium Instance:
Best assignment: (0, 0, 0, 1, 1)
Minimum number of bins used: 2

Testing Large Instance:


To solve the QUBO, use quantum annealing simulators. You can use the Dwave Ocean Framework. Here is an example.

In [ ]:
def solve_qubo(weights, capacity):
    qubo_matrix = ilp_to_qubo(weights, capacity)
    
    # Convert QUBO matrix to a dictionary
    qubo_dict = {}
    for i in range(len(qubo_matrix)):
        for j in range(len(qubo_matrix)):
            if qubo_matrix[i, j] != 0:
                qubo_dict[(i, j)] = qubo_matrix[i, j]

    # Set up the D-Wave sampler
    sampler = EmbeddingComposite(DWaveSampler())

    # Solve the QUBO problem
    sampleset = sampler.sample_qubo(qubo_dict, num_reads=100)

    return sampleset

In [ ]:
# Test instances
test_cases = {
    "Small": ([1, 2, 3], 4),
    "Medium": ([2, 5, 3, 7, 1], 10),
    "Large": ([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 20)
}

# Run quantum annealing simulator for each test case
for name, (weights, capacity) in test_cases.items():
    print(f"\nTesting {name} Instance:")
    solution = solve_qubo(weights, capacity)
    print("Quantum Annealing Solution:")
    print(solution)

Use a Quantum Variational approach to solve the QUBO. 

* Create multiple Ansantz for tests. 
* Build a function with input being the QUBO and Ansantz. Using a hybrid approach solved the QUBO. 

I have used Qiskit framework.


In [ ]:
# create multiple ansazts to test
def create_ansatz(num_qubits):
    ansatz = EfficientSU2(num_qubits, entanglement='linear')
    return ansatz

In [ ]:
# hybrid approach to solve the QUBO and a simulator
def variational_solver(qubo_matrix, ansatz, num_qubits):
    # Create a quantum instance
    backend = Aer.get_backend('aer_simulator')
    quantum_instance = QuantumInstance(backend, shots=1024)

    # Setup VQE
    vqe = VQE(ansatz, optimizer=SLSQP(), quantum_instance=quantum_instance)

    # Define the Hamiltonian based on the QUBO
    hamiltonian = qubo_to_hamiltonian(qubo_matrix)

    # Run VQE
    result = vqe.compute_minimum_eigenvalue(hamiltonian)

    return result

In [ ]:
def qubo_to_hamiltonian(qubo_matrix):
    num_vars = qubo_matrix.shape[0]
    terms = []
    
    for i in range(num_vars):
        for j in range(num_vars):
            coeff = qubo_matrix[i, j]
            if coeff != 0:
                terms.append((coeff, Pauli.from_list([(i, 'Z'), (j, 'Z')])))
    
    return sum(coeff * Pauli.from_list([(i, 'Z'), (j, 'Z')]) for coeff, (i, j) in terms)

In [ ]:
# Test instances
test_cases = {
    "Small": ([1, 2, 3], 4),
    "Medium": ([2, 5, 3, 7, 1], 10),
    "Large": ([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 20)
}

# Run VQE for each test case
for name, (weights, capacity) in test_cases.items():
    print(f"\nTesting {name} Instance:")
    qubo_matrix = ilp_to_qubo(weights, capacity)
    ansatz = create_ansatz(qubo_matrix.shape[0])
    
    result = variational_solver(qubo_matrix, ansatz, qubo_matrix.shape[0])
    print("Optimal value:", result.eigenvalue)

## Use QAOA to solve the QUBO. 

* Create from scratch a QAOA function


In [ ]:
def create_qaoa_circuit(num_qubits, p, gamma, beta):
    # Create a quantum circuit with the required number of qubits
    qc = QuantumCircuit(num_qubits)

    # Apply the Hadamard gate to all qubits
    qc.h(range(num_qubits))

    # Apply the QAOA layers
    for layer in range(p):
        # Apply the problem unitary
        for i in range(num_qubits):
            qc.rz(2 * gamma[layer], i)  # Rotation around Z-axis
            if i < num_qubits - 1:
                qc.cx(i, i + 1)  # CNOT between adjacent qubits

        # Apply the mixer unitary
        for i in range(num_qubits):
            qc.rx(2 * beta[layer], i)  # Rotation around X-axis

    return qc

In [ ]:
def expectation_value(qc, qubo_matrix, backend):
    # Simulate the circuit to get the counts
    counts = backend.run(qc).result().get_counts()
    expectation = 0

    # Calculate the expectation value based on counts
    for outcome, count in counts.items():
        outcome_value = 0
        for i in range(len(outcome)):
            if outcome[i] == '1':
                outcome_value += 1  # Adjust for your specific QUBO matrix
        expectation += count * (outcome_value)

    return expectation / sum(counts.values())

In [ ]:
def optimize_qaoa(qubo_matrix, p):
    num_qubits = qubo_matrix.shape[0]
    backend = Aer.get_backend('aer_simulator')

    # Define parameters for the QAOA circuit
    gamma = [Parameter(f'γ_{i}') for i in range(p)]
    beta = [Parameter(f'β_{i}') for i in range(p)]

    # Objective function to minimize
    def objective_function(params):
        gammas = params[:p]
        betas = params[p:]
        qc = create_qaoa_circuit(num_qubits, p, gammas, betas)
        return expectation_value(qc, qubo_matrix, backend)

    # Initial guess for gamma and beta
    initial_params = np.random.rand(2 * p)

    # Minimize the objective function
    result = minimize(objective_function, initial_params, method='COBYLA')

    return result

In [ ]:
# Run QAOA for each test case
for name, (weights, capacity) in test_cases.items():
    print(f"\nTesting {name} Instance:")
    qubo_matrix = ilp_to_qubo(weights, capacity)
    p = 1  # Number of layers in QAOA
    result = optimize_qaoa(qubo_matrix, p)
    print("Optimal parameters:", result.x)
    print("Minimum value:", result.fun)